In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/snli/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%tensorflow_version 2.x
!pip install tensorflow-addons
!pip install transformers

In [3]:
from transformers import RobertaTokenizer, TFRobertaModel

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.2.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Enabled: True


In [4]:
params = {
  'train_path': '../data/train.txt',
  'test_path': '../data/test.txt',
  'pretrain_path': 'roberta-base',
  'num_samples': 550152,
  'buffer_size': 200000,
  'batch_size': 32,
  'max_len': 128 + 3,
  'num_patience': 5,
  'init_lr': 1e-5,
  'max_lr': 3e-5,
}

In [5]:
tokenizer = RobertaTokenizer.from_pretrained(params['pretrain_path'],
                                             lowercase = True,
                                             add_special_tokens = True)

In [6]:
# stream data from text files
def data_generator(f_path, params):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      text1 = tokenizer.tokenize(text1)
      text2 = tokenizer.tokenize(text2)
      if len(text1) + len(text2) + 3 > params['max_len']:
        _max_len = (params['max_len'] - 3) // 2
        text1 = text1[:_max_len]
        text2 = text2[:_max_len]
      text = ['<s>'] + text1 + ['</s>'] + text2 + ['</s>']
      text = tokenizer.convert_tokens_to_ids(text)
      yield text, label2idx[label]


def dataset(is_training, params):
  _shapes = ([None], ())
  _types = (tf.int32, tf.int32)
  _pads = (1, -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [7]:
# input stream ids check
text, _ = next(data_generator(params['train_path'], params))
print(text)

Reading ../data/train.txt
[0, 102, 621, 15, 10, 5253, 13855, 81, 10, 3187, 159, 16847, 2, 102, 621, 16, 1058, 39, 5253, 13, 10, 1465, 2]


In [8]:
class RobertaFinetune(tf.keras.Model):
  def __init__(self, params):
    super(RobertaFinetune, self).__init__()
    self.bert = TFRobertaModel.from_pretrained(params['pretrain_path'],
                                               trainable = True)
    self.drop_1 = tf.keras.layers.Dropout(.1)
    self.fc = tf.keras.layers.Dense(300, tf.nn.swish, name='down_stream/fc')
    self.drop_2 = tf.keras.layers.Dropout(.1)
    self.out = tf.keras.layers.Dense(3, name='down_stream/out')

  def call(self, bert_inputs, training):
    bert_inputs = [tf.cast(inp, tf.int32) for inp in bert_inputs]
    x = self.bert(bert_inputs, training=training)[1]
    x = self.drop_1(x, training=training)
    x = self.fc(x)
    x = self.drop_2(x, training=training)
    x = self.out(x)
    return x

In [ ]:
model = RobertaFinetune(params)
model.build([[None, None], [None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

step_size = 2 * params['num_samples'] // params['batch_size']
decay_lr = tfa.optimizers.Triangular2CyclicalLearningRate(
  initial_learning_rate = params['init_lr'],
  maximal_learning_rate = params['max_lr'],
  step_size = step_size,)
optim = tf.optimizers.Adam(params['init_lr'])
global_step = 0

best_acc = .0
count = 0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for (text, labels) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      masks = tf.cast(tf.math.not_equal(text, 1), tf.int32)
      logits = model([text, masks], training=True)
      loss = tf.compat.v1.losses.softmax_cross_entropy(
        tf.one_hot(labels, 3, dtype=tf.float32),
        logits = logits,
        label_smoothing = .2,)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, 5.)
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 100 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for (text, labels) in dataset(is_training=False, params=params):
    masks = tf.cast(tf.math.not_equal(text, 1), tf.int32)
    logits = model([text, masks], training=False)
    m.update_state(y_true=labels, y_pred=tf.argmax(logits, -1))

  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))

  if acc > best_acc:
    best_acc = acc
    # you can save model here
    count = 0
  else:
    count += 1
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if count == params['num_patience']:
    print(params['num_patience'], "times not improve the best result, therefore stop training")
    break

Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


[('tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/kernel:0',
  TensorShape([768, 768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/bias:0',
  TensorShape([768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/kernel:0',
  TensorShape([768, 768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/bias:0',
  TensorShape([768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/kernel:0',
  TensorShape([768, 768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/bias:0',
  TensorShape([768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/output/dense/kernel:0',
  TensorShape([768, 768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/output/dense/bias:0',
  TensorShape([768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/output/LayerNorm/gamma:0',
  TensorShape([768])),
 ('tf_roberta_model/roberta/encoder/layer_._0/attention/output/LayerNorm/b